# Read router output and display selected trajectories

In [2]:
import os, sys, gzip, random, csv, json, datetime, re
import pymongo
import pandas as pd
import folium
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as funcs
from pyspark.sql import SQLContext
from pyspark import Row
from pyspark.sql.functions import udf
from  pyspark.sql.types import BooleanType
with open('geomadi_pub.json') as f:
    cred = json.load(f)
baseDir = "./"

AttributeError: 'module' object has no attribute 'SSL_ST_INIT'

In [2]:
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark-client"
conf = SparkConf()
conf.setMaster('yarn-client')
conf.setAppName('ttrajectory_counts')
conf.set("spark.executor.memory", "8g")
conf.set("spark.executor.cores", "8")
conf.set("spark.executor.instances", "8")

In [3]:
#sc.stop()
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

## Read Parquet File - load nodes

In [4]:
import json
jobF = json.load(open("qsm.odm_extraction.odm_via_via.json","r"))
locL = pd.DataFrame(jobF['odm_via_conf']['input_locations'])
locL.loc[:,"id"] = locL['node_list'].apply(lambda x: x[0])
nodeL = list(locL[locL['location_id']=='entry_44']['node_list'])[0]
nodeL

[266186942, 1579223006]

In [7]:
#a_df = sqlContext.read.parquet("/tdg/2017/09/12/paths_krauthausen")
#a_df = sqlContext.read.parquet("/tdg/2017/09/12/experiments/paths_ab_routing_full")
a_df = sqlContext.read.parquet("/tdg/2017/05/23/experiments/paths_ab_routing_full_test")
a_df.show(1)

Py4JJavaError: An error occurred while calling o142.parquet.
: java.io.FileNotFoundException: File hdfs://cro-out-test-master1.priv.motionlogic.de:8020/tdg/2017/09/12/experiments/paths/path_krauthausen/part-r-00000-e8234988-11e0-4284-9dbb-1860951a5b8e.gz.parquet does not exist.
	at org.apache.hadoop.hdfs.DistributedFileSystem.listStatusInternal(DistributedFileSystem.java:923)
	at org.apache.hadoop.hdfs.DistributedFileSystem.access$600(DistributedFileSystem.java:114)
	at org.apache.hadoop.hdfs.DistributedFileSystem$22.doCall(DistributedFileSystem.java:985)
	at org.apache.hadoop.hdfs.DistributedFileSystem$22.doCall(DistributedFileSystem.java:981)
	at org.apache.hadoop.fs.FileSystemLinkResolver.resolve(FileSystemLinkResolver.java:81)
	at org.apache.hadoop.hdfs.DistributedFileSystem.listStatus(DistributedFileSystem.java:992)
	at org.apache.spark.sql.sources.HadoopFsRelation$FileStatusCache$$anonfun$1.apply(interfaces.scala:458)
	at org.apache.spark.sql.sources.HadoopFsRelation$FileStatusCa

## Filter out trajectories on via nodes

In [122]:
def filter_via_node(nodeL): 
    '''fitler trajectories by via nodes'''
    def meth(trajectory):
        node_ids = set([node_tuple[0] for node_tuple in trajectory])
        for node in nodeL:
            if node in node_ids:
                return True
        return False
    return meth

In [123]:
check_trajectory = udf(filter_via_node(nodeL), BooleanType()) ##filter trajectories by node
trSel = a_df.filter(check_trajectory(a_df.Trajectory))
trajL = trSel.select("Trajectory").collect()

In [124]:
if False:
    count = trSel.select('IMSI').distinct().count() ##count unique trajectories
    print(count)
print(len(trajL))

3041


## Display a sample trajectory

In [145]:
trajN = 358
trajP = pd.DataFrame({"node":[x[0] for x in trajL[trajN][0]],"ts":[x[1] for x in trajL[trajN][0]]})
trajP.sort_values("ts",inplace=True)
print(trajP.head(10))
Time = ((int(trajP["ts"].tail(1))-int(trajP["ts"].head(1)))/60)/60
print("Overall Time in Hours:", Time)

         node          ts
0  2156402180  1505182920
1  2156402181  1505193774
2  2951492644  1505193779
3   309370264  1505193784
4   299798937  1505193788
5   266186956  1505193793
6   266186935  1505193798
7  1579223006  1505193803
8  1579223036  1505193807
9  1579222985  1505193812
('Overall Time in Hours:', 23)


In [126]:
client = pymongo.MongoClient(cred['mongo']['address'],cred['mongo']['port'])         #connect to mongo
coll = client["tdg_infra_18a04_required_nodes"]["nodes"]

In [139]:
coord = []
nodeQ = coll.find({"node_id":{"$in":list(trajP['node'].values)}})
for n in nodeQ:
    coord.append({"x":n["loc"]["coordinates"][1],
                  "y":n["loc"]["coordinates"][0],
                  "node":n["node_id"]})
coorD = pd.DataFrame(coord) 
print(coorD.head(1))

     node          x       y
0  485951  50.892798  10.948


In [140]:
junctL = []
nodeQ = coll.find({"node_id":{"$in":nodeL}})
for n in nodeQ:
    junctL.append({"x":n["loc"]["coordinates"][1],
                  "y":n["loc"]["coordinates"][0],
                  "node":n["node_id"]})
junctL = pd.DataFrame(junctL)  

# Plot trajectories in Folium 
#### Green - Start
#### Red - End
#### Blue - Ramp

In [141]:
mapp = folium.Map(location=[51.133,10.288],
                  zoom_start = 5
                  )  #Central Germany

folium.features.Circle(location=[junctL['x'][0],junctL['y'][0]] ## junction
                       ,popup="entry_44",radius=3000,color="blue",fill=True,fill_color="white").add_to(mapp)

folium.features.Circle(location=list(coorD.head(1)[['x','y']].values[0]) ## start
                       ,popup="start",radius=2000,color="green",fill=True,fill_color="white").add_to(mapp)

folium.features.Circle(location=list(coorD.tail(1)[['x','y']].values[0]) ## end
                       ,popup="end",radius=2000,color="red",fill=True,fill_color="white").add_to(mapp)

for i, g in coorD.iterrows(): ## nodes
    folium.features.Circle(radius=20,location=[g["x"],g["y"]],color='black').add_to(mapp)
    
#folium.PolyLine([coorD["x"],coorD["y"]],color="blue",weight=1.5,opacity=1).add_to(mapp)
mapp